[The author's git-->](https://github.com/angrigoriev)

In [3]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
from IPython.display import clear_output

In [4]:
from sklearn.metrics import roc_auc_score, mean_squared_error

In [5]:
import pickle

In [7]:
# обзор финальных данных
df_all_profiles = pickle.load(open('df_all_profiles','rb'))
df_genres = pickle.load(open('df_genres.pcl','rb'))
kino_genres = pickle.load(open('paid_genres','rb'))
df_all_profiles_kino = pickle.load(open('df_all_profiles_scnm','rb'))
df_rating = pickle.load(open('df_rating','rb'))

In [8]:
df_all_profiles.head(n=2)

,purchase_count,0,1,2,3,4,5,6,7,8,...,94,95,96,97,98,99,100,101,102,user_id
user_id,,,,,,,,,,,,,,,,,,,,,
1654,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1654
510087,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,510087


In [9]:
df_genres.head(n=2)

,item_id,content_type,0,1,2,3,4,5,6,7,...,93,94,95,96,97,98,99,100,101,102
item_id,,,,,,,,,,,,,,,,,,,,,
326,326,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
336,336,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
kino_genres.head(n=2)

,item_id,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,add_data
0,326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,336,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
df_all_profiles_kino.head(n=2)

,user_id,0,0,1,2,3,4,5,6,7,...,19,20,21,22,23,24,25,26,27,28
0,1654,-0.040688,0.0,0.0,0.408248,0.000000,0.000000,0.408248,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.408248,0.000000,0.000000
1,510087,0.025179,0.0,0.0,0.213201,0.213201,0.213201,0.213201,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.426401,0.213201,0.213201


In [12]:
df_rating.head(n=2)

,item_id,rating
0,326,6.173
1,336,4.147


In [37]:
def prepare_XY(df, add=True):
    df_all_profiles = pickle.load(open('df_all_profiles','rb'))
    df_genres = pickle.load(open('df_genres.pcl','rb'))
    df = df.merge(df_genres, how='left', on='item_id')
    df = df.merge(df_all_profiles, how='left', on='user_id')
    del df_genres
    del df_all_profiles

    if True:
        kino_genres = pickle.load(open('paid_genres','rb'))
        df_all_profiles_kino = pickle.load(open('df_all_profiles_scnm','rb'))
        df_rating = pickle.load(open('df_rating','rb'))
        df = df.merge(kino_genres, how='left', on='item_id')
        df = df.merge(df_rating, how='left', on='item_id')
        df = df.merge(df_all_profiles_kino, how='left', on='user_id')
        del kino_genres
        del df_all_profiles_kino
    Y = np.array([y for y in df['purchase']])
    df.drop(['purchase','user_id','item_id','content_type'],axis=1,inplace=True)
    
    return (df.values, Y)

In [111]:
def submit(model,epoch=None,save_only=False):
    df_test = pd.read_csv('lab10data/lab10_test.csv',sep=',')
    X_test, Y_test = prepare_XY(df_test,True)

    probas = model.predict_proba(X_test, batch_size=8000) if hasattr(model, 'predict_proba') else model.predict(X_test, batch_size=8000)
    df_test['purchase'] = 1 - probas
    df_test.sort_values(['user_id','item_id'], inplace=True)
    if save_only:
        df_test.to_csv('lab10s.csv_epoch_' + epoch,index=False)
    else:
        df_test.to_csv('lab10s.csv',index=False)

    del df_test
    del X_test
    del Y_test
    del probas

In [38]:
df_train = pd.read_csv('lab10data/lab10_train.csv',sep=',')

In [39]:
X_train, Y_train = prepare_XY(df_train, True)

In [40]:
del df_train

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
skf = StratifiedKFold(n_splits=100)

In [10]:
skf.get_n_splits(X_train, Y_train)

100

In [11]:
train_index, test_index = next(skf.split(X_train, Y_train))

In [41]:
xtr = X_train[train_index,:]
xts = X_train[test_index,:]
ytr = Y_train[train_index]
yts = Y_train[test_index]

In [42]:
del X_train, Y_train

In [15]:
import keras

Using TensorFlow backend.


In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

In [109]:
class CountROCAUC(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:
            test_predicted_labels = self.model.predict_proba(xts, batch_size=16384)
            print('roc_auc',epoch, roc_auc_score(yts,test_predicted_labels))

In [114]:
class SaveModel(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 50 == 0:
            model.save('nn_epoch_' + str(epoch))
            submit(model,str(epoch),True)

In [43]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import BatchNormalization

activation='tanh'
model = Sequential()
model.add(Dense(256, input_shape=(xtr.shape[1],))) 
model.add(Activation(activation))
model.add(Dropout(0.5))

model.add(Dense(128)) 
model.add(Activation(activation))
model.add(Dropout(0.5))

model.add(Dense(64)) 
model.add(Activation(activation))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')

In [115]:
countROC = CountROCAUC()
saveModel = SaveModel()
model.fit(xtr, ytr,epochs=2000,batch_size=16000,validation_data=(xts,yts), callbacks=[countROC,saveModel],class_weight='balance')

Train on 4982296 samples, validate on 50328 samples
Epoch 1/2000
4982296/4982296 [==============================] - 47s - loss: 0.0096 - val_loss: 0.0371
Epoch 2/2000
4982296/4982296 [==============================] - 16s - loss: 0.0096 - val_loss: 0.0363
Epoch 3/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0377
Epoch 4/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0393
Epoch 5/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0386
Epoch 6/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0415
Epoch 7/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0365
Epoch 8/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0378
Epoch 9/2000
4982296/4982296 [==============================] - 15s - loss: 0.0096 - val_loss: 0.0375
Epoch 10/2000
4982296/4982296 

In [116]:
submit(model)

2144000/2156840 [============================>.] - ETA: 0s

In [117]:
!scp lab10s.csv vb:lab10s.csv